In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("../")
from algorithm.koopmanlib.dictionary import PsiNN
from algorithm.koopmanlib.target import NonLinearPendulum # Duffing data comes from target.py, not other data source

In [ ]:
basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=61)

In [ ]:
# Generate data
pendulum_train = NonLinearPendulum(n_init=1000, traj_len=1000, seed=0)
data_x_train = pendulum_train.generate_init_data()
data_y_train = pendulum_train.generate_next_data(data_x_train)

pendulum_valid = NonLinearPendulum(n_init=300, traj_len=1000, seed=0)
data_x_valid = pendulum_valid.generate_init_data()
data_y_valid = pendulum_valid.generate_next_data(data_x_valid)

In [ ]:
# # Visualize data
# fig, (ax1,ax2) = plt.subplots(1,2,sharey=True, figsize=(10,5))

# ax1.plot(data_x_train[:,0], data_x_train[:,1], '.')
# ax1.set_xlabel(r'$x_{1}$')
# ax1.set_ylabel(r'$x_{2}$')
# ax2.plot(data_x_valid[:,0], data_x_valid[:,1], '.')
# ax2.set_xlabel(r'$x_{1}$')
# ax2.set_ylabel(r'$x_{2}$')
# plt.tight_layout()
# Visualize data
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 5))

ax1.plot(data_x_train[:, 0], data_x_train[:, 1], '.')
ax1.set_xlabel(r'$x_{1}$')
ax1.set_ylabel(r'$x_{2}$')
ax1.set_title('Training Data')

ax2.plot(data_x_valid[:, 0], data_x_valid[:, 1], '.')
ax2.set_xlabel(r'$x_{1}$')
ax2.set_ylabel(r'$x_{2}$')
ax2.set_title('Validation Data')

plt.tight_layout()
plt.show()


In [ ]:
data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

In [ ]:
from algorithm.koopmanlib.solver_3 import KoopmanDLSolver
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=2,
                         reg=0.1)

In [ ]:
solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=50,
             batch_size=5000,
             lr=1e-4,
             log_interval=20,
             lr_decay_factor=0.8)

In [ ]:
# Results from solver3
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(data_x_train)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs3 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("pendulum_demo_outputs_solver3.npy", resDMD_DL_outputs3, allow_pickle=True)

print(resDMD_DL_outputs3['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs3['resDMD_DL_outputs']['efuns'].shape)

In [ ]:
A = np.array(Psi_X).T @ np.array(Psi_Y)

In [ ]:
A.shape

In [ ]:
N1 = 50
N2 = 100

Id1 = np.maximum(np.outer(np.abs(np.arange(-1*N1, N1+1)), np.ones(N2)), 1)
Id2 = np.outer(np.ones(2*N1+1), np.arange(1, N2+1))
# N_trun = N2
N_trun = 10

Id = np.where(np.abs(Id1.flatten()) * np.abs(Id2.flatten()) < N_trun + 1)
A = A[Id][:, Id]  # sort out indexing
A = A.reshape((Id[0].shape[0],Id[0].shape[0]))
# A = A.reshape((964,964))

# Compute pseudospectra
x_pts = np.arange(-1.5, 1.5, 0.05)
y_pts = x_pts
z_pts = np.outer(x_pts, np.ones(len(y_pts))) + 1j * np.outer(np.ones(len(x_pts)), y_pts)
z_pts = z_pts.flatten()




In [ ]:
import sys
sys.path.insert(0, 'C:\\Program Files\\MATLAB\\R2023a\\extern\\engines\\python')
import matlab.engine
eng = matlab.engine.start_matlab()
# Add the directory containing KoopPseudoSpec.m to MATLAB's path
eng.addpath('D:\\Residual-Dynamic-Mode-Decomposition-main\\main_routines', nargout=0)

RES, _, _ = eng.KoopPseudoSpec(np.eye(len(A)), A, np.eye(len(A)), z_pts,nargout=3)
# RES = KoopPseudoSpec(np.eye(len(A)), A, np.eye(len(A)), z_pts)
RES = RES.reshape(len(y_pts), len(x_pts))

print(type(RES))
print(len(RES))

# RES = np.array(RES)
RES = np.asarray(RES)
print(RES.shape)

In [ ]:
from scipy.linalg import eigh, eig
# EDMD eigenvalues
E = eig(A)[0]

In [ ]:
# Create a figure
plt.figure()

# Your MATLAB code uses a logarithmic scale for coloring based on the values of 1./real(RES),
# and specifies custom contour levels based on log10(1./v)
v = np.array([1e-50, 0.25])  # v array as specified in your MATLAB code
contour_levels = np.log10(1.0 / v)  # Calculate contour levels
contour_levels = sorted(list(contour_levels))
# Reshape real and imaginary parts of z_pts
real_z_reshaped = np.reshape(np.real(z_pts), (len(y_pts), len(x_pts)))
imag_z_reshaped = np.reshape(np.imag(z_pts), (len(y_pts), len(x_pts)))

# Get log10 of 1./real(RES)
log_res = np.log10(1.0 / np.real(RES))

# Create a filled contour plot
contour = plt.contourf(real_z_reshaped, imag_z_reshaped, log_res, levels=contour_levels, colors='#999999')

# Set the linewidth of the contour lines
for c in contour.collections:
    c.set_linewidth(1.5)
plt.gca().set_aspect('equal', 'box')
plt.plot(E.real, E.imag, '.m')
plt.plot(np.exp(1j * np.linspace(0, 2*np.pi, 1000)).real, np.exp(1j * np.linspace(0, 2*np.pi, 1000)).imag, '-r')
# Display the plot
plt.show()